In [27]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle


try:
    model = tf.keras.models.load_model('my_model2.keras')
    print("Model loaded successfully")
except Exception as e:
    print("Error loading model:", e)


with open('saved_objects.pkl', 'rb') as file:
    aa_properties = pickle.load(file)
properties_dict = aa_properties['properties_dict']

def encode_with_properties(prot_primary_seq, properties_dict):
    encoded_seqs = []
    for sequence in prot_primary_seq:
        encoded_seq = []
        for aa in sequence:
            if aa in properties_dict:
                encoded_seq.append(properties_dict[aa])
            else:
                encoded_seq.append([0.0] * len(next(iter(properties_dict.values()))))  # Default to zeros if unknown
        encoded_seqs.append(encoded_seq)
    return encoded_seqs


sequence = 'MSEYIRVTEDENDEPIEIPSEDDGTVLLSTVTAQFPGACGLRYRNPVSQCMRGVRLVEGILHAPDAGWGNLVYVVNYPKDNKRKMDETDASSAVKVKRAVQKTSDLIVLGLPWKTTEQDLKEYFSTFGEVLMVQVKKDLKTGHSKGFGFVRFTEYETQVKVMSQRHMIDGRWCDCKLPNSKQSQDEPLRSRKVFVGRCTEDMTEDELREFFSQYGDVMDVFIPKPFRAFAFVTFADDQIAQSLCGEDLIIKGISVHISNAEPKHNSNRQLERSGRFGGNPGGFGNQGGFGNSRGGGAGLGNNQGSNMGGGMNFGAFSINPAMMAAAQAALQSSWGMMGMLASQQNQSGPSGNNQNQGNMQREPNQAFGSGNNSYSGSNSGAAIGWGSASNAGSGSGFNGGFGSSMDSKSSGWGM'
encoded_sequence = encode_with_properties([sequence], properties_dict)
max_length = len(sequence)  
padded_sequence = pad_sequences(encoded_sequence, maxlen=max_length, padding='post', dtype='float32')


padded_sequence = np.array(padded_sequence)
if len(padded_sequence.shape) == 2:
    padded_sequence = np.expand_dims(padded_sequence, axis=0)


print(f"Encoded sequence shape: {np.shape(encoded_sequence)}")
print(f"Padded sequence shape: {padded_sequence.shape}")


predicted_structure = model.predict(padded_sequence)


classes = ['H', 'B', 'T']
predicted_classes = np.argmax(predicted_structure, axis=-1)
predicted_secondary_structure = [classes[pred] for pred in predicted_classes[0]]


predicted_secondary_structure_string = ''.join(predicted_secondary_structure)

print("Predicted Secondary Structure:", predicted_secondary_structure_string)


C:\Users\Guillaume\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 8 variables whereas the saved optimizer has 14 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


Model loaded successfully
Encoded sequence shape: (1, 414, 7)
Padded sequence shape: (1, 414, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 508ms/step
Predicted Secondary Structure: BBBBHHHHBHBHHBHHHBHBHBBBHHHHHHBBBBBBHBBHBHBHBHBBBHBBHBHHBBHBHHHBBBHBHBHBHBBBBBBBBHHHBHBHBHBHBBHBHBHBHHHBBBHBHBBHBHBBBHBBHHHBHBBHBBHBBHBBHHBHBHHHBHBHBBHHBBBHBBBHBHHBBHBBBBHBHBBHTBHHBHBHBBBBHBBHHBBHBHBHBBBBHBBHHBHHBBHHBBBBHHBBHBHHBHBHHBBBBHBHBHHHBBBHBBHHBBHBBHBHBBHBBHHBHHBBHBHHHBHHHBBHBBHHBHBHHHBBBHBHBHBBBHHBBHBHBHBBHHBBBBHBHBBBBBBBHBHBBHBHHBHBBBBBBHBBBBBBBHBBHHBBBHBHBHBHHBBHBBHBBBBHBHBHBBBHBBBHBBBHBHBBBHBHBBHHHT
